# Notebook Template

This Notebook is stubbed out with some project paths, loading of enviroment variables, and common package imports to speed up the process of starting a new project.

It is highly recommended you copy and rename this notebook following the naming convention outlined in the readme of naming notebooks with a double number such as `01_first_thing`, and `02_next_thing`. This way the order of notebooks is apparent, and each notebook does not need to be needlesssly long, complex, and difficult to follow.

In [10]:
import importlib
import os
from pathlib import Path
import sys

from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.gis import GIS
from dotenv import load_dotenv, find_dotenv
import pandas as pd
import janitor

# import arcpy if available
if importlib.util.find_spec("arcpy") is not None:
    import arcpy
    
# load environment variables from .env
load_dotenv(find_dotenv())

[autoreload of unacast failed: Traceback (most recent call last):
  File "C:\Program Files\ArcGIS\Pro\bin\Python\envs\covid-19-unacast\lib\site-packages\IPython\extensions\autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "C:\Program Files\ArcGIS\Pro\bin\Python\envs\covid-19-unacast\lib\site-packages\IPython\extensions\autoreload.py", line 434, in superreload
    module = reload(module)
  File "C:\Program Files\ArcGIS\Pro\bin\Python\envs\covid-19-unacast\lib\imp.py", line 315, in reload
    return importlib.reload(module)
  File "C:\Program Files\ArcGIS\Pro\bin\Python\envs\covid-19-unacast\lib\importlib\__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 618, in _exec
  File "<frozen importlib._bootstrap_external>", line 674, in exec_module
  File "<frozen importlib._bootstrap_external>", line 781, in get_code
  File "<frozen importlib._bootstrap_external>", line 741, in source_to_code
  

True

In [11]:
# create a GIS object instance; if you did not enter any information here, it defaults to anonymous access to ArcGIS Online
gis = GIS(
    url=os.getenv('ESRI_GIS_URL'), 
    username=os.getenv('ESRI_GIS_USERNAME'),
    password=os.getenv('ESRI_GIS_PASSWORD')
)

# paths to common data locations - NOTE: to convert any path to a raw string, simply use str(path_instance)
project_parent = Path('./').absolute().parent

data_dir = project_parent/'data'

data_raw = data_dir/'raw'
data_ext = data_dir/'external'
data_int = data_dir/'interim'
data_out = data_dir/'processed'

gdb_raw = data_raw/'raw.gdb'
gdb_int = data_int/'interim.gdb'
gdb_out = data_out/'processed.gdb'

# import the project package from the project package path
sys.path.append(str(project_parent/'src'))
import unacast

# load the "autoreload" extension so that code can change, & always reload modules so that as you change code in src, it gets loaded
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
unacast_csv_pth = data_raw/'covid_sds_full_2020-03-30.csv'
itm_id = '7566e0221e5646f99ea249a197116605'

In [20]:
df = pd.read_csv(unacast_csv_pth).rename_column('date', 'localeventdate')
df.head()

,localeventdate,weekday,covid,state_fips,state_name,state_code,county_fips,county_name,county_population,daily_distance_diff,daily_visitation_diff,grade_total,grade_distance,grade_visitation,n_grade_total,n_grade_distance,n_grade_visitation,last_updated
0,2020-02-28,6,PRE,1,Alabama,AL,1001,Autauga County,55601,0.017884,-0.055463,F,F,F,1.0,1,1.0,2020-03-30 09:45:31.219313 UTC
1,2020-02-24,2,PRE,1,Alabama,AL,1001,Autauga County,55601,-0.041471,-0.118980,F,F,F,1.0,1,1.0,2020-03-30 09:45:31.219313 UTC
2,2020-03-18,4,POST,1,Alabama,AL,1001,Autauga County,55601,-0.026997,-0.223407,F,F,F,1.0,1,1.0,2020-03-30 09:45:31.219313 UTC
3,2020-03-25,4,POST,1,Alabama,AL,1001,Autauga County,55601,-0.137809,-0.314639,F,F,F,1.0,1,1.0,2020-03-30 09:45:31.219313 UTC
4,2020-03-22,1,POST,1,Alabama,AL,1001,Autauga County,55601,-0.291821,-0.468586,D-,D,F,1.5,2,1.0,2020-03-30 09:45:31.219313 UTC


In [23]:
unacast.create_unacast_feature_class(itm_id, unacast_csv_pth, gdb_int/'unacast')

'D:\\projects\\covid-19-unacast\\data\\interim\\interim.gdb\\unacast'

In [28]:
county_srs = unacast.get_county_geometry_series(itm_id)
una_df = unacast.load_unacast_csv(unacast_csv_pth)

In [46]:
full_df = una_df.join(county_srs, on='county_fips').rename_column('date', 'localeventdate')
full_df.head()

,localeventdate,weekday,covid,state_fips,state_name,state_code,county_fips,county_name,county_population,daily_distance_diff,daily_visitation_diff,grade_total,grade_distance,grade_visitation,n_grade_total,n_grade_distance,n_grade_visitation,last_updated,SHAPE
0,2020-02-28,6,PRE,1,Alabama,AL,01001,Autauga County,55601,0.017884,-0.055463,F,F,F,1.0,1,1.0,2020-03-30 09:45:31.219313+00:00,"{'rings': [[[-86.8206698922349, 32.34730961636..."
1,2020-02-24,2,PRE,1,Alabama,AL,01001,Autauga County,55601,-0.041471,-0.118980,F,F,F,1.0,1,1.0,2020-03-30 09:45:31.219313+00:00,"{'rings': [[[-86.8206698922349, 32.34730961636..."
2,2020-03-18,4,POST,1,Alabama,AL,01001,Autauga County,55601,-0.026997,-0.223407,F,F,F,1.0,1,1.0,2020-03-30 09:45:31.219313+00:00,"{'rings': [[[-86.8206698922349, 32.34730961636..."
3,2020-03-25,4,POST,1,Alabama,AL,01001,Autauga County,55601,-0.137809,-0.314639,F,F,F,1.0,1,1.0,2020-03-30 09:45:31.219313+00:00,"{'rings': [[[-86.8206698922349, 32.34730961636..."
4,2020-03-22,1,POST,1,Alabama,AL,01001,Autauga County,55601,-0.291821,-0.468586,D-,D,F,1.5,2,1.0,2020-03-30 09:45:31.219313+00:00,"{'rings': [[[-86.8206698922349, 32.34730961636..."


In [47]:
output_feature_class = gdb_int/'unacast'
if arcpy.Exists(str(output_feature_class)):
    arcpy.management.Delete(str(output_feature_class))
full_df.spatial.to_featureclass(output_feature_class)

'D:\\projects\\covid-19-unacast\\data\\interim\\interim.gdb\\unacast'

In [49]:
day_df = full_df[full_df.localeventdate == full_df.localeventdate.max()]
day_df.head()

,localeventdate,weekday,covid,state_fips,state_name,state_code,county_fips,county_name,county_population,daily_distance_diff,daily_visitation_diff,grade_total,grade_distance,grade_visitation,n_grade_total,n_grade_distance,n_grade_visitation,last_updated,SHAPE
11,2020-03-26,5,POST,1,Alabama,AL,01001,Autauga County,55601,-0.194783,-0.433452,F,F,F,1.0,1,1.0,2020-03-30 09:45:31.219313+00:00,"{""rings"": [[[-86.8206698922349, 32.34730961636..."
55,2020-03-26,5,POST,1,Alabama,AL,01003,Baldwin County,218022,-0.286008,-0.604218,C-,D,C,2.5,2,3.0,2020-03-30 09:45:31.219313+00:00,"{""rings"": [[[-87.9730898104582, 31.16482247822..."
92,2020-03-26,5,POST,1,Alabama,AL,01005,Barbour County,24881,-0.069718,-0.185546,F,F,F,1.0,1,1.0,2020-03-30 09:45:31.219313+00:00,"{""rings"": [[[-85.7433728247864, 31.62624048792..."
121,2020-03-26,5,POST,1,Alabama,AL,01007,Bibb County,22400,-0.139756,-0.423225,F,F,F,1.0,1,1.0,2020-03-30 09:45:31.219313+00:00,"{""rings"": [[[-87.4198556163033, 33.01177309252..."
140,2020-03-26,5,POST,1,Alabama,AL,01009,Blount County,57840,-0.210837,-0.339984,F,F,F,1.0,1,1.0,2020-03-30 09:45:31.219313+00:00,"{""rings"": [[[-86.9679945194109, 33.86045468586..."


In [52]:
len(day_df.county_fips.unique()), len(county_srs.index.unique())

(3054, 3142)

In [1]:
df.pivot(index='county_fips', columns='localeventdate', values=['GRADE', 'N_GRADE', 'daily_distance_diff'])

NameError: name 'df' is not defined